In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pyspiel
import open_spiel.python.examples.ubc_utils
from open_spiel.python.visualizations import ubc_treeviz
from open_spiel.python.examples.ubc_utils import *
from auctions.webutils import *
from absl import logging
logging.set_verbosity(logging.INFO)
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator


In [2]:
# load runs from experiments
experiments = []
experiments += ['jun27_port']
runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
print(f"Found {len(runs)} runs")

Found 223 runs


In [5]:
def draw_game_tree(game, policy, fname, figure_dir='figures/2023-06-27-graphviz'):
    node_policy_decorator, edge_policy_decorator = ubc_treeviz.make_policy_decorators(policy)
    gametree = ubc_treeviz.GameTree(
        game,
        node_decorator=node_policy_decorator,
        edge_decorator=edge_policy_decorator,
        group_infosets=False,
        group_terminal=False,
        group_pubsets=False, 
        target_pubset='*',
        depth_limit=20,
        state_prob_limit=0.001,
        action_prob_limit=0.01, 
        policy=policy
    )
    
    outfile = os.path.join(figure_dir, fname)
    gametree.draw(outfile, prog='dot')
    print("Game tree saved to file", outfile)
    

In [6]:
for run in tqdm(runs[:50]):
    game, checkpoint, _ = get_results(run, load_policy=False)
    env_and_policy = ppo_db_checkpoint_loader(checkpoint)
    for player in range(game.num_players()):
        env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
    policy = env_and_policy.make_policy()

    draw_game_tree(
        game, 
        policy, 
        fname=f'{checkpoint}-{checkpoint.game.name}.pdf'.replace('/', '_').replace(' ', '_'),
        figure_dir='figures/2023-06-28-graphviz'
    )
    

  4%|██████▋                                                                                                                                                               | 2/50 [00:01<00:24,  1.97it/s]

Game tree saved to file figures/2023-06-28-graphviz/jun26_jun26_4_tie_break-cfr_port_2external_plus-101_(jun28_port)_Iteration_994-jun26_jun26_4_tie_break.json.pdf
Game tree saved to file figures/2023-06-28-graphviz/jun26_jun26_0_base-cfr_port_2outcome_lin_avg_plus-100_(jun28_port)_Iteration_3237895-jun26_jun26_0_base.json.pdf


  6%|█████████▉                                                                                                                                                            | 3/50 [00:01<00:24,  1.89it/s]

Game tree saved to file figures/2023-06-28-graphviz/jun26_jun26_1_grace_period-cfr_port_2outcome_lin_avg-100_(jun28_port)_Iteration_2200001-jun26_jun26_1_grace_period.json.pdf


  8%|█████████████▎                                                                                                                                                        | 4/50 [00:03<00:46,  1.00s/it]

...

Game tree saved to file figures/2023-06-28-graphviz/jun26_jun26_3_grace_period-cfr_port_2outcome_lin_avg_plus-102_(jun28_port)_Iteration_2280001-jun26_jun26_3_grace_period.json.pdf


 96%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍      | 48/50 [00:41<00:04,  2.16s/it]

Game tree saved to file figures/2023-06-28-graphviz/jun26_jun26_1_grace_period-cfr_port_2external_plus-100_(jun28_port)_Iteration_86-jun26_jun26_1_grace_period.json.pdf


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:41<00:00,  1.20it/s]

Game tree saved to file figures/2023-06-28-graphviz/jun26_jun26_3_base-cfr_port_2outcome_lin_avg-101_(jun28_port)_Iteration_3601815-jun26_jun26_3_base.json.pdf
Game tree saved to file figures/2023-06-28-graphviz/jun26_jun26_3_grace_period-cfr_port_2outcome_lin_avg-100_(jun28_port)_Iteration_2805053-jun26_jun26_3_grace_period.json.pdf


In [4]:
def find_run_by_graphviz_name(target_fname):
    for run in tqdm(runs[:50]):
        game, checkpoint, _ = get_results(run, load_policy=False)
        fname = f'{checkpoint}-{checkpoint.game.name}.pdf'.replace('/', '_').replace(' ', '_')

        if fname == target_fname:
            break
    return game, checkpoint

In [5]:
game, checkpoint = find_run_by_graphviz_name(
    'jun26_jun26_2_tie_break-cfr_port_2outcome_lin_avg_plus-101_(jun27_port)_Iteration_27119705-jun26_jun26_2_tie_break.json.pdf'
)

        
# env_and_policy = ppo_db_checkpoint_loader(checkpoint, env_params=EnvParams(history_prefix=[0, 1, 7, 3, 5], include_state=True))
# env_and_policy = ppo_db_checkpoint_loader(checkpoint, env_params=EnvParams(include_state=True))
env_and_policy = ppo_db_checkpoint_loader(checkpoint)
# for player in range(game.num_players()):
#     env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
policy = env_and_policy.make_policy()


 18%|█████████████████████████████▉                                                                                                                                        | 9/50 [00:00<00:00, 87.95it/s]


In [6]:
draw_game_tree(
    game, 
    policy, 
    fname=f'non_modal_test.pdf'
)

Game tree saved to file figures/2023-06-27-graphviz/non_modal_test.pdf


In [215]:
agent = env_and_policy.agents[1]
ts = env_and_policy.env.reset()

In [275]:
state_0 = game.new_initial_state().child(0).child(1).child(5).child(6)
p0_policy = policy.action_probabilities(state_0)

In [276]:
tremble_prob = 0.25
for a in p0_policy:
    p0_policy[a] = tremble_prob / len(p0_policy) + (1 - tremble_prob) * p0_policy[a]

In [277]:
p0_policy

{0: 0.05097339022596157,
 1: 0.0509253221554808,
 2: 0.051117293302417495,
 4: 0.2610026394114132,
 5: 0.5859813549047269}

In [278]:
state_1 = game.new_initial_state().child(0).child(1).child(5).child(6).child(5)
state_1.information_state_string() == 'p1t1 r2 posted[[32.0, 19.0], [41.6, 19.0]] sub[[0, 0], [1, 2]] a70 agg[[0, 0], [2, 3]] proc[[0, 0], [1, 2]]'
p1_policy = policy.action_probabilities(state_1)

In [279]:
def get_exp_reward(p1_action):
    p1_exp_reward = 0
    for p0_action in p0_policy:
        child_state = state_0.child(p0_action).child(p1_action)
        if child_state.is_terminal():
            p1_exp_reward += p0_policy[p0_action] * child_state.returns()
        elif child_state.is_chance_node():
            for chance_action_id, chance_prob in child_state.chance_outcomes():
                if child_state.child(chance_action_id).is_terminal():
                    p1_exp_reward += p0_policy[p0_action] * chance_prob * child_state.child(chance_action_id).returns()
                else:
                    print(f"auction didn't end with actions ({p0_action}, {p1_action})")
                    p1_exp_reward += p0_policy[p0_action] * chance_prob * (-23)
    return p1_exp_reward


In [280]:
print(get_exp_reward(0))
print(get_exp_reward(1))
# print(get_exp_reward(2))

[117.48991219  28.56168062]
[117.48991219  28.56168062]


In [265]:
p1_policy

{0: 0.4178864348496503,
 1: 0.2165796062193167,
 2: 0.22909362982280757,
 3: 0.009561295284364005,
 4: 0.05347208393775575,
 5: 0.039379712758246956,
 6: 0.034027237127858455}

In [25]:
game, checkpoint = find_run_by_graphviz_name(
    'jun26_jun26_0_base-cfr_port_2outcome_lin_avg-102_(jun27_port)_Iteration_32734471-jun26_jun26_0_base.json.pdf'
)
env_and_policy = ppo_db_checkpoint_loader(checkpoint)
# for player in range(game.num_players()):
#     env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
policy = env_and_policy.make_policy()

  2%|███▎                                                                                                                                                                 | 1/50 [00:00<00:00, 146.29it/s]


In [89]:
other_checkpoint = checkpoint.equilibrium_solver_run.equilibriumsolverruncheckpoint_set.order_by('t')[4]
env_and_policy = ppo_db_checkpoint_loader(other_checkpoint)
# for player in range(game.num_players()):
#     env_and_policy.agents[player] = ModalAgentDecorator(env_and_policy.agents[player])
policy = env_and_policy.make_policy()

In [90]:
other_checkpoint.get_modal_eval().player_improvements

[0.0, 2.267499999999984]

In [91]:
state = game.new_initial_state().child(1).child(0).child(5).child(3)
state.information_state_string() == 'p0t1 r2 posted[[32.0, 19.0], [41.6, 19.0]] sub[[0, 0], [1, 2]] a70 agg[[0, 0], [2, 2]] proc[[0, 0], [1, 2]]'

policy.action_probabilities(state)

{0: 0.17162356456899808,
 1: 0.11744036472904104,
 2: 0.10892114269603584,
 3: 0.17021437116411217,
 4: 0.14597193111736637,
 5: 0.28582862572444634}

In [43]:
regrets, avg_policy = policy._agents[0].policy._infostates[state.information_state_string()]

In [44]:
regrets

array([-5.36012297e+15, -5.81796428e+15,  7.15178971e+06])

In [45]:
avg_policy

array([5.33644809e+03, 1.28490232e+02, 3.97179908e+08])

In [33]:
regrets / regrets.sum()

array([-0.33288927,  0.21847515,  0.34070933,  0.18763323,  0.40632068,
        0.17975089])

In [34]:
avg_policy / avg_policy.sum()

array([0.17162356, 0.11744036, 0.10892114, 0.17021437, 0.14597193,
       0.28582863])

In [ ]:
agent = policy._agents[0]
infostates = agent.policy._infostates

In [ ]:
state = game.new_initial_state().child(1).child(0).child(4).child(4)

In [80]:
state.information_state_string(0)

'p0t1 r2 posted[[32.0, 19.0], [41.6, 19.0]] sub[[0, 0], [1, 0]] a32 agg[[0, 0], [2, 0]] proc[[0, 0], [1, 0]]'

In [81]:
game.ob

AttributeError: 'ClockAuctionGame' object has no attribute 'ob'

In [82]:
agent.policy.action_probabilities(state)

{0: 0.026900200506226456, 1: 0.02273779054313965, 4: 0.9503620089506339}

In [51]:
for k in infostates:
    
    print(k, infostates[k][1])
    break

p0t1 r1 posted[[32, 19]] sub[[0, 0]] a89 agg[[0, 0]] proc[[0, 0]] [2.11579216e+01 2.04997729e+01 1.02332616e+03 1.22727141e+05
 5.71808349e+07 9.28859416e+05 4.41867890e+05 4.77753707e+05]
